## Chapter 2. Introduction to TensorFlow Extended



*   The TensorFlow Extended (TFX) library has all the components for a machine learning pipeline.
*   Pipeline can be executed using an orchestrator (Airflow or Kubeflow Pipelines)



Apache Beam is an open source tool for defining and executing data-processing jobs:
1.   runs under the hood of many TFX components to carry out processing steps like data validation or data preprocessing
2.   can be used as a pipeline orchestrator


In 2019, Google then published the open source glue code containing all the required pipeline components to tie the libraries together and to automatically create pipeline definitions for orchestration tools like Apache Airflow, Apache Beam, and Kubeflow Pipelines.

*   Data ingestion with ExampleGen
*   Data validation with StatisticsGen, SchemaGen, and the ExampleValidator
*   Data preprocessing with Transform
*   Model training with Trainer
*   Checking for previously trained models with ResolverNode
*   Model analysis and validation with Evaluator
*   Model deployments with Pusher

In [ ]:
#install TFX
$ pip install tfx

In [ ]:
#load packages
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam


#or import TFX component
from tfx.components import ExampleValidator
from tfx.components import Evaluator
from tfx.components import Transform

Internal Components of a Machine Learning Pipeline Component
1.   Driver: receives input by querying the metadata store
2.   Executor: performs the action of the component
3.   Publisher: Saves the output or final result metadata in the MetadataStore

Artifacts - inputs and outputs. Ex: raw input data, preprocessed data, and trained models



*   The components of TFX “communicate” through metadata; instead of passing artifacts directly between the pipeline components, the components consume and publish references to pipeline artifacts.
*   One advantage of passing the metadata between components instead of the direct artifacts is that the information can be centrally stored.


Currently, MLMD supports three types of backends:
1.   In-memory database (via SQLite)
2.   SQLite
3.   MySQL


In [ ]:
#import required packages
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext


#create a context object which handles component execution and displays the component’s artifacts
context = InteractiveContext()


#execute each component object
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)


#inspect features of the dataset
context.show(statistics_gen.outputs['statistics'])


#access artifact properties
for artifact in statistics_gen.outputs['statistics'].get():
    print(artifact.uri)

Alternatives to TFX

*   AirBnb - Aerosolve
*   Stripe - Railyard
*   Spotify - Luigi
*   Uber - Micheangelo
*   Netflix - Metaflow

"Apache Beam offers you an open source, vendor-agnostic way to describe data processing steps that then can be executed on various environments. Since it is incredibly versatile, Apache Beam can be used to describe batch processes, streaming operations, and data pipelines. In fact, TFX relies on Apache Beam and uses it under the hood for a variety of components (e.g., TensorFlow Transform or TensorFlow Data Validation)."

In [ ]:
#install Apache Beam
$ pip install apache-beam

#install Apache Beam for Google Cloud
$ pip install 'apache-beam[gcp]'

#Install Apache Beam for AWS
$ pip install 'apache-beam[boto]'

In [ ]:
#Apache Beam Introduction

import re

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

input_file = "gs://dataflow-samples/shakespeare/kinglear.txt"   #text is stored in a Google Cloud bucket
output_file = "/tmp/output.txt"

# Define pipeline options object.
pipeline_options = PipelineOptions()

with beam.Pipeline(options=pipeline_options) as p:  #set up the apache beam pipeline
    # Read the text file or file pattern into a PCollection.
    lines = p | ReadFromText(input_file)  #create a data collection by reading the textfile

    # Count the occurrences of each word.
    counts = (  #perform the transformations on the collection
        lines
        | 'Split' >> beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x))
        | 'PairWithOne' >> beam.Map(lambda x: (x, 1))
        | 'GroupAndSum' >> beam.CombinePerKey(sum))

    # Format the counts into a PCollection of strings.
    def format_result(word_count):
        (word, count) = word_count
        return "{}: {}".format(word, count)

    output = counts | 'Format' >> beam.Map(format_result)

    # Write the output using a "Write" transform that has side effects.
    output | WriteToText(output_file)

In [ ]:
#If you want to execute this pipeline on different Apache Beam runners like Apache Spark or Apache Flink, 
#you will need to set the pipeline configurations through the pipeline_options object

python basic_pipeline.py